In [ ]:
import pandas as pd

df_updated = pd.read_csv("BLIP_results_facial_expressions.csv")

In [ ]:
df_correct = df_updated[df_updated["correct_answer"] == df_updated["generated_answer"]]
df_correct

In [ ]:
def is_correct_first(row):
    prompt = row['clean_prompt']
    correct_answer = row['correct_answer']
    return 'feeling ' + correct_answer + ' or' in prompt

# Adding the 'correct_first' column
df_correct['correct_first'] = df_correct.apply(is_correct_first, axis=1)


In [ ]:
df_correct['correct_answer'].unique()

In [ ]:
def sample_emotions(df, emotion, sample_size=250):
    correct_first_true = df[(df['correct_answer'] == emotion) & (df['correct_first'] == True)]
    correct_first_false = df[(df['correct_answer'] == emotion) & (df['correct_first'] == False)]
    
    # Ensure we don't sample more than available
    sample_true = correct_first_true.sample(n=min(sample_size, len(correct_first_true)), random_state=1)
    sample_false = correct_first_false.sample(n=min(sample_size, len(correct_first_false)), random_state=1)
    
    return pd.concat([sample_true, sample_false])

emotions = df_correct['correct_answer'].unique()

samples = pd.concat([sample_emotions(df_correct, emotion) for emotion in emotions])


In [ ]:
emotion_counts = samples['correct_answer'].value_counts()
correct_first_counts = samples['correct_first'].value_counts()


In [ ]:
emotion_counts

In [ ]:
correct_first_counts

In [ ]:
len(samples)

In [ ]:
samples.to_csv("BLIP_final_facial_expressions.csv", index=False)

# Creating corrupted prompts

In [ ]:
df = pd.read_csv("BLIP_final_facial_expressions.csv")

In [ ]:
# Mapping core adjectives to "positive", "negative", and "neutral"
core_to_sentiment = {
    "angry": "negative",
    "disgusted": "negative",
    "fearful": "negative",
    "happy": "positive",
    "neutral": "neutral",
    "sad": "negative",
    "surprised": "neutral"
}

# Initial adjective to core mapping
adjective_to_core = {
    "amused": "happy",
    "ashamed": "sad",
    "awed": "surprised",
    "bored": "neutral",
    "calm": "neutral",
    "confident": "happy",
    "confused": "neutral",
    "content": "happy",
    "curious": "neutral",
    "disoriented": "neutral",
    "distracted": "neutral",
    "embarrassed": "sad",
    "envious": "angry",
    "flustered": "fearful",
    "giggly": "happy",
    "grateful": "happy",
    "grumpy": "angry",
    "hopeful": "happy",
    "horrified": "fearful",
    "insecure": "fearful",
    "jealous": "angry",
    "lonely": "sad",
    "nostalgic": "sad",
    "overwhelmed": "fearful",
    "pleased": "happy",
    "puzzled": "neutral",
    "relaxed": "happy",
    "relieved": "happy",
    "restless": "neutral",
    "satisfied": "happy",
    "scornful": "disgusted",
    "shocked": "surprised",
    "shy": "neutral",
    "sincere": "neutral",
    "startled": "surprised",
    "stressed": "fearful",
    "thoughtful": "neutral",
    "uncomfortable": "disgusted",
    "apathetic": "neutral",
    "impressed": "surprised",
    "guilty": "sad",
    "humiliated": "sad",
    "sympathetic": "neutral"
}

# Create the final dictionary with sentiment classification
adjective_to_core_and_sentiment = {
    adj: [core, core_to_sentiment[core]]
    for adj, core in adjective_to_core.items()
}

print(adjective_to_core_and_sentiment)


In [ ]:
import random
sentiment_to_adjectives = {
    "positive": [adj for adj, core in adjective_to_core.items() if core_to_sentiment[core] == "positive"],
    "negative": [adj for adj, core in adjective_to_core.items() if core_to_sentiment[core] == "negative"],
    "neutral": [adj for adj, core in adjective_to_core.items() if core_to_sentiment[core] == "neutral"]
}

# Function to replace adjectives in the prompt
def replace_adjective(prompt):
    parts = prompt.split("feeling ")[1].split(" or ")
    first, second = parts[0], parts[1].rstrip("?")

    # Determine the sentiment of the original adjectives
    first_sentiment = core_to_sentiment.get(adjective_to_core.get(first, first), "neutral")
    second_sentiment = core_to_sentiment.get(adjective_to_core.get(second, second), "neutral")

    # Select opposite adjectives
    if first_sentiment == "positive":
        new_first = random.choice(sentiment_to_adjectives["negative"] + sentiment_to_adjectives["neutral"])
    else:
        new_first = random.choice(sentiment_to_adjectives["positive"])

    if second_sentiment == "positive":
        new_second = random.choice(sentiment_to_adjectives["negative"] + sentiment_to_adjectives["neutral"])
    else:
        new_second = random.choice(sentiment_to_adjectives["positive"])

    return f"Is this person feeling {new_first} or {new_second}?"

df["corrupt_prompt"] = df["clean_prompt"].apply(replace_adjective)
df["corrupt_prompt"]

In [ ]:
df.to_csv("BLIP_final_facial_expressions.csv", index=False)